In [1]:
import os
%pwd

'e:\\Food_Delivery\\notebooks'

In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_path: Path
    test_path: Path
    preprocessor_model_path: Path
    model_path:Path

In [4]:
from src.constants import *
from src.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_path=config.train_path,
            test_path=config.test_path,
            preprocessor_model_path=config.preprocessor_model_path,
            model_path=config.model_path
        )

        return model_trainer_config

In [6]:
import os
from src.logging import logger
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle

from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
        
    def initiate_model_training(self):
        train = pd.read_csv(self.config.train_path)
        test = pd.read_csv(self.config.test_path)
        
        X_train = train.drop('Time_taken(min)', axis = 1)
        X_test = test
        y_train = train['Time_taken(min)']
        
        preprocessor_path = self.config.preprocessor_model_path
        
        with open(preprocessor_path, 'rb') as file:
            preprocessor = pickle.load(file)
            
        
        X_train = preprocessor.fit_transform(X_train)
        X_test = preprocessor.transform(X_test)
        
        
        rf = RandomForestRegressor()
        
        rf.fit(X_train, y_train)
        
        rf_model = self.config.model_path
        
        logger.info(f"Saving model.pkl file at {rf_model}")
        
        with open(rf_model, 'wb') as file:
            pickle.dump(rf, file)
        

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.initiate_model_training()
except Exception as e:
    raise e

[2023-08-22 20:57:57,976: INFO: __init__: yaml_file: config\config.yaml loaded succesfully...]
[2023-08-22 20:57:57,979: INFO: __init__: yaml_file: params.yaml loaded succesfully...]
[2023-08-22 20:57:57,980: INFO: __init__: Directory created at: artifacts]
[2023-08-22 20:57:57,981: INFO: __init__: Directory created at: artifacts/models]
